In [11]:
import torch
import pandas as pd
import torch.nn as nn

test_df = pd.read_csv('./data/test_df.csv', index_col=0)
means = pd.read_csv('./data/means.csv', index_col = 0)
stds = pd.read_csv('./data/stds.csv', index_col = 0)

y_cols = ["Målt konsistens [mm]", "Målt densitet [kg/m3]", "Målt luftinnhold [%]", "Sylinder 28 døgn"]

X_test_tensor = torch.tensor(test_df.drop(columns=y_cols).values, dtype=torch.float32)
y_test_tensor = torch.tensor(test_df[y_cols].values, dtype=torch.float32)

print("test data loaded successfully")


class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, output_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.exp(x)
        return x

# Define the path to the saved model
model_path = './models/simple_nn_model.pth'

# Load the model
model = SimpleNN(X_test_tensor.shape[1], y_test_tensor.shape[1])
model.load_state_dict(torch.load(model_path))
model.eval() 

print("Model loaded successfully.")


y_stds_tensor = torch.tensor(stds.loc[y_cols].to_numpy(), dtype=torch.float32)
y_means_tensor = torch.tensor(means.loc[y_cols].to_numpy(), dtype=torch.float32)
X_stds_tensor = torch.tensor(stds.drop(index=y_cols).to_numpy(), dtype=torch.float32)
X_means_tensor = torch.tensor(means.drop(index=y_cols).to_numpy(), dtype=torch.float32)

def denormalizeInputs(inputs):
    return inputs.view(-1, 1) * X_stds_tensor + X_means_tensor

def denormalizeOutputs(outputs):
    return outputs.view(-1, 1) * y_stds_tensor + y_means_tensor

    

test data loaded successfully
Model loaded successfully.


In [12]:
material_costs_df = pd.read_csv("./data/material_costs_estimates.csv", index_col=0).drop(columns=["Category"])
material_costs_tensor = torch.tensor(material_costs_df.to_numpy(), dtype=torch.float32)

def materials_cost(inputs):
    return (inputs * material_costs_tensor).sum()


In [ ]:
def objective_function(inputs, doPrint=False, min_strength = 60):
    
    outputs = denormalizeOutputs(model(inputs))
    strength = outputs[3, -1]
    cost = materials_cost(denormalizeInputs(inputs))

    if (doPrint):
        print("STRENGTH:", strength.item(), "COST:", cost.item())

    objective = torch.where(strength < min_strength, strength, -(cost ** 2) / 1.7e9                            )
    return objective

In [ ]:
# DEBUGGING

# inputs = torch.zeros(1, X_test_tensor.shape[1], requires_grad=True)
# objective_function(inputs, doPrint=True)

STRENGTH: 55.236175537109375 COST: 322037.75


tensor(55.2362, grad_fn=<WhereBackward0>)

In [ ]:

SAMPLES = 20
ITERATIONS = 1500
MIN_STRENGTH = 61

for i in range(SAMPLES):

    inputs = torch.normal(mean=0, std=1, size=(1, X_test_tensor.shape[1]), requires_grad=True)

    best_inputs = inputs
    lowest_cost = materials_cost(denormalizeInputs(inputs))


    for j in range(ITERATIONS):

        outputs = model(inputs)

        # Compute gradients without backward()
        doPrint = j == ITERATIONS -1
        # doPrint = False
        grads = torch.autograd.grad(objective_function(inputs, doPrint=doPrint, min_strength=MIN_STRENGTH), inputs, grad_outputs=torch.ones_like(objective_function(inputs)), retain_graph=True)[0]

        inputs = torch.clamp(inputs + grads * 0.01, min=-(X_means_tensor / (X_stds_tensor - 1e-8)).view(1, -1))

        # print("OUTPUT:", denormalizeOutputs(model(inputs)))
    
    cost = materials_cost(denormalizeInputs(inputs))
    if (cost < lowest_cost):
        lowest_cost = cost
        print("NEW BEST:")
        objective_function(inputs, doPrint=True, min_strength=MIN_STRENGTH)
        
    # print(denormalizeInputs(inputs))
    # print(denormalizeOutputs(outputs))



STRENGTH: 60.15398025512695 COST: 25519.673828125
STRENGTH: 60.00508117675781 COST: 32491.751953125
STRENGTH: 60.0704460144043 COST: 30538.046875
STRENGTH: 60.082374572753906 COST: 30916.794921875
STRENGTH: 60.204071044921875 COST: 27246.9296875
STRENGTH: 60.19423294067383 COST: 28803.703125
STRENGTH: 60.44723129272461 COST: 31958.083984375
STRENGTH: 60.07618713378906 COST: 27487.9609375
STRENGTH: 60.35517120361328 COST: 28674.04296875
STRENGTH: 60.23454666137695 COST: 30937.390625
STRENGTH: 60.070350646972656 COST: 29628.279296875
STRENGTH: 60.16218185424805 COST: 27697.609375
STRENGTH: 60.20504379272461 COST: 32112.806640625
STRENGTH: 60.31995391845703 COST: 26142.451171875
STRENGTH: 60.036903381347656 COST: 32631.28125
STRENGTH: 60.35343933105469 COST: 26150.537109375
STRENGTH: 60.4276008605957 COST: 33203.47265625
STRENGTH: 61.690406799316406 COST: 32421.0546875
STRENGTH: 60.01220703125 COST: 29848.0703125
STRENGTH: 60.525291442871094 COST: 30348.94140625
